--- Day 19: A Series of Tubes ---

Somehow, a network packet got lost and ended up here. It's trying to follow a routing diagram (your puzzle input), but it's confused about where to go.

Its starting point is just off the top of the diagram. Lines (drawn with |, -, and +) show the path it needs to take, starting by going down onto the only line connected to the top of the diagram. It needs to follow this path until it reaches the end (located somewhere within the diagram) and stop there.

Sometimes, the lines cross over each other; in these cases, it needs to continue going the same direction, and only turn left or right when there's no other option. In addition, someone has left letters on the line; these also don't change its direction, but it can use them to keep track of where it's been. For example:

```
     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
```

Given this diagram, the packet needs to take the following path:

- Starting at the only line touching the top of the diagram, it must go down, pass through A, and continue onward to the first +.
- Travel right, up, and right, passing through B in the process.
- Continue down (collecting C), right, and up (collecting D).
- Finally, go all the way left through E and stopping at F.

Following the path to the end, the letters it sees on its path are `ABCDEF`.

The little packet looks up at you, hoping you can help it find the way. **What letters will it see** (in the order it would see them) if it follows the path? (The routing diagram is very wide; make sure you view it without line wrapping.)

In [8]:
library(stringr)
test_route <- readr::read_lines("test_input.txt")
route <- readr::read_lines("input.txt")

In [9]:
make_grid <- function(route_lines) {
  # route_lines should be a chr vector of each line of the routes text
  # make_grid creates a list of lines representing coordiates starting at (1,1) and increasing going right and down
  # therefore make_grid with coordinates (5, 15) would represent the 5th line and 15th character in that line
  width <- max(unlist(lapply(route_lines, nchar)))
  new_lines <- str_pad(string = route_lines, width, side = "right")
  split_lines <- str_split(new_lines, "")
  return(split_lines)
}

In [10]:
start_position <- function(route_grid) {
  # route_grid is a list of lines, each row containing the text of the paths available
  not_spaces <- !(route_grid[[1]] == " ")
  possibilities <- 1:length(not_spaces)
  first_non_space <- min(possibilities[not_spaces])
  start_val <- route_grid[[1]][first_non_space]
  direction <- if (start_val == "|") {
    "down"
  } else if (start_val == "-") {
    "right"
  }
  position <- list(row = 1, col = first_non_space, direction = direction, 
                   letters_collected = c(), steps_count = 0, stop_going = FALSE)
  
  return(position)
}

In [11]:
next_position <- function(route_grid, 
                          current_position = list(row = 1, col = 1, 
                                                  direction = "down", 
                                                  letters_collected = c(),
                                                  steps_count = 0,
                                                  stop_going = FALSE)
                          ) {
  # next_position takes the current position and direction in a grid and determines what the next position will be
  steps_count <- current_position$steps_count + 1
  letters_collected <- current_position$letters_collected
  current_direction <- current_position$direction
  if (current_direction == "up") {
    next_pos <- list(row = current_position$row - 1, col = current_position$col)
  } else if (current_direction == "down") {
    next_pos <- list(row = current_position$row + 1, col = current_position$col)
  } else if (current_direction == "left") {
    next_pos <- list(row = current_position$row, col = current_position$col - 1)
  } else if (current_direction == "right") {
    next_pos <- list(row = current_position$row, col = current_position$col + 1)
  }
  
  val_at_cur_pos <- route_grid[[current_position$row]][current_position$col]
  val_at_next_pos <- route_grid[[next_pos$row]][next_pos$col]
  if (val_at_next_pos %in% LETTERS) {
    letters_collected <- append(letters_collected, val_at_next_pos)
    new_direction <- current_position$direction
  } else if (val_at_next_pos == "+") {
    steps_count <- steps_count + 1
    # decide what new direction and position are
    if (current_direction == "down") {
      val_check_left <- tryCatch(route_grid[[next_pos$row]][next_pos$col - 1],
                                 error = function(e) 0)
      val_check_right <- tryCatch(route_grid[[next_pos$row]][next_pos$col + 1],
                                  error = function(e) 0)
      if (val_check_left == "-") {
        new_direction <- "left"
        next_pos <- list(row = next_pos$row, col = next_pos$col - 1)
      } else if (val_check_right == "-") {
        new_direction <- "right"
        next_pos <- list(row = next_pos$row, col = next_pos$col + 1)
      } else if (val_check_left %in% LETTERS) {
        new_direction <- "left"
        next_pos <- list(row = next_pos$row, col = next_pos$col - 1)
        letters_collected <- append(letters_collected, val_check_left)
      } else if (val_check_right %in% LETTERS) {
        new_direction <- "right"
        next_pos <- list(row = next_pos$row, col = next_pos$col + 1)
        letters_collected <- append(letters_collected, val_check_right)
      }
    }
    if (current_direction == "up") {
      val_check_left <- tryCatch(route_grid[[next_pos$row]][next_pos$col - 1],
                                 error = function(e) 0)
      val_check_right <- tryCatch(route_grid[[next_pos$row]][next_pos$col + 1],
                                  error = function(e) 0)
      if (val_check_left == "-") {
        new_direction <- "left"
        next_pos <- list(row = next_pos$row, col = next_pos$col - 1)
      } else if (val_check_right == "-") {
        new_direction <- "right"
        next_pos <- list(row = next_pos$row, col = next_pos$col + 1)
      } else if (val_check_left %in% LETTERS) {
        new_direction <- "left"
        next_pos <- list(row = next_pos$row, col = next_pos$col - 1)
        letters_collected <- append(letters_collected, val_check_left)
      } else if (val_check_right %in% LETTERS) {
        new_direction <- "right"
        next_pos <- list(row = next_pos$row, col = next_pos$col + 1)
        letters_collected <- append(letters_collected, val_check_right)
      }
    }
    if (current_direction == "left") {
      val_check_up <- tryCatch(route_grid[[next_pos$row - 1]][next_pos$col],
                               error = function(e) 0)
      val_check_down <- tryCatch(route_grid[[next_pos$row + 1]][next_pos$col],
                                 error = function(e) 0)
      if (val_check_up == "|") {
        new_direction <- "up"
        next_pos <- list(row = next_pos$row - 1, col = next_pos$col)
      } else if (val_check_down == "|") {
        new_direction <- "down"
        next_pos <- list(row = next_pos$row + 1, col = next_pos$col)
      } else if (val_check_up %in% LETTERS) {
        new_direction <- "up"
        next_pos <- list(row = next_pos$row - 1, col = next_pos$col)
        letters_collected <- append(letters_collected, val_check_up)
      } else if (val_check_down %in% LETTERS) {
        new_direction <- "down"
        next_pos <- list(row = next_pos$row + 1, col = next_pos$col)
        letters_collected <- append(letters_collected, val_check_down)
      }
    }
    if (current_direction == "right") {
      val_check_up <- tryCatch(route_grid[[next_pos$row - 1]][next_pos$col],
                               error = function(e) 0)
      val_check_down <- tryCatch(route_grid[[next_pos$row + 1]][next_pos$col],
                                 error = function(e) 0)
      if (val_check_up == "|") {
        new_direction <- "up"
        next_pos <- list(row = next_pos$row - 1, col = next_pos$col)
      } else if (val_check_down == "|") {
        new_direction <- "down"
        next_pos <- list(row = next_pos$row + 1, col = next_pos$col)
      } else if (val_check_up %in% LETTERS) {
        new_direction <- "up"
        next_pos <- list(row = next_pos$row - 1, col = next_pos$col)
        letters_collected <- append(letters_collected, val_check_up)
      } else if (val_check_down %in% LETTERS) {
        new_direction <- "down"
        next_pos <- list(row = next_pos$row + 1, col = next_pos$col)
        letters_collected <- append(letters_collected, val_check_down)
      }
    }
  } else {
    new_direction <- current_position$direction
  }
  if (val_at_next_pos == " ") {
    end_grid <- TRUE
  } else {
    end_grid <- FALSE
  }
  
  return(list(row = next_pos$row, col = next_pos$col, 
              direction = new_direction, 
              letters_collected = letters_collected,
              steps_count = steps_count,
              stop_going = end_grid))
}

In [12]:
test_grid <- make_grid(test_route)
test_start <- start_position(test_grid)
new_pos <- next_position(test_grid, test_start)
at_end <- new_pos$stop_going
while(!at_end) {
  new_pos <- next_position(test_grid, new_pos)
  at_end <- new_pos$stop_going
}
new_pos

$row
[1] 4

$col
[1] 1

$direction
[1] "left"

$letters_collected
[1] "A" "B" "C" "D" "E" "F"

$steps_count
[1] 38

$stop_going
[1] TRUE

In [13]:
my_grid <- make_grid(route)
my_start <- start_position(my_grid)
new_pos <- next_position(my_grid, my_start)
at_end <- new_pos$stop_going
while(!at_end) {
  new_pos <- next_position(my_grid, new_pos)
  at_end <- new_pos$stop_going
}
new_pos

$row
[1] 31

$col
[1] 16

$direction
[1] "up"

$letters_collected
[1] "G" "I" "N" "O" "W" "K" "Y" "X" "H"

$steps_count
[1] 16636

$stop_going
[1] TRUE

In [14]:
paste0(new_pos$letters_collected, collapse = "")

[1] "GINOWKYXH"

**The first half of this puzzle is complete! It provides one gold star: \* **

--- Part Two ---

The packet is curious how many steps it needs to go.

For example, using the same routing diagram from the example above...

```
     |          
     |  +--+    
     A  |  C    
 F---|--|-E---+ 
     |  |  |  D 
     +B-+  +--+ 
```

...the packet would go:

- 6 steps down (including the first line at the top of the diagram).
- 3 steps right.
- 4 steps up.
- 3 steps right.
- 4 steps down.
- 3 steps right.
- 2 steps up.
- 13 steps left (including the F it stops on).

This would result in a total of 38 steps.

How many steps does the packet need to go?

In [15]:
new_pos$steps_count

[1] 16636

That's the right answer! You are one gold star closer to debugging the printer.